In [ ]:
import os

In [ ]:
###Pick the Model that we would like to train on

#model_id = "stabilityai/stable-diffusion-3-medium"
#model_id = "stabilityai/stable-diffusion-3-medium-diffusers"
model_id = "stabilityai/stable-diffusion-3.5-medium"
prompt_text = "a photo of sks dog at the beach"
dimension = 512

In [ ]:
#install the latest diffusers libraries and scripts
!git clone https://github.com/huggingface/diffusers

In [ ]:
#install additional libraries required for SD3
os.chdir("diffusers")
!pip install -e .

In [ ]:
#install dreambooth to help with training
os.chdir("examples/dreambooth")
!pip install -r requirements_sd3.txt > sd3_install.log

In [ ]:
#create a hugging face session which allows us to download gated models
my_token = get_generic_secret_string("my_hf_token")
from huggingface_hub import login(token=my_token)


In [ ]:
from huggingface_hub import snapshot_download

local_dir = "./dog"
snapshot_download(
    "diffusers/dog-example",
    local_dir=local_dir,
    repo_type="dataset",
    ignore_patterns=".gitattributes",
)

In [ ]:
os.chdir("dog")
!rm -r .cache
os.chdir('..')

In [ ]:
#create a config file for accelerate. accelerate is a hugging face library to simplify training
from accelerate.utils import write_basic_config
write_basic_config()

#this config is for a single medium GPU pool, but you can adjust as needed for your workload
yaml_content = """
compute_environment: LOCAL_MACHINE
distributed_type: MULTI_GPU
num_processes: 4
num_machines: 1
machine_rank: 0
use_cpu: false
mixed_precision: "bf16"
main_training_function: main
downcast_bf16: 'no'
"""
with open('/root/.cache/huggingface/accelerate/default_config.yaml', 'w') as file:
#with open('accelerate_config.yaml', 'w') as file:
    file.write(yaml_content)



In [ ]:
#this is the accelerate script that allows us to pass general parameters for fine tuning
!accelerate launch train_dreambooth_lora_sd3.py \
  --pretrained_model_name_or_path="{model_id}" \
  --instance_data_dir="dog" \
  --output_dir="./path_to_saved_model" \
  --gradient_checkpointing \
  --mixed_precision="bf16" \
  --instance_prompt="a photo of sks dog" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=500 \
  --seed="42424242"  > output.log 2>&1

In [ ]:
#in case you want to see errors/progress of your training. 
#once you see success in the log, you know your model is ready
#note: downloading models takes time!
!cat output.log

In [ ]:
#create a snowflake session in this notebook, we can use this to access functionality in snowflake
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
# we can save the model to our stage with a unique name
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%Y_%m_%d_%H%M%S")

put_result = session.file.put("./path_to_saved_model/pytorch_lora_weights.safetensors","@artifacts/models/SD3Med_MyModel_"+dt_string, auto_compress= False)
put_result[0].status